# Get Team Data
This notebook pulls team level data to compare and predict winners.

In [76]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime

from importlib import reload
import projections
reload(projections)

pd.options.display.max_colwidth = 200
pd.options.display.max_rows, pd.options.display.min_rows = 35, 35

In [91]:
current_season = 2023
url = f'https://www.pro-football-reference.com/years/{current_season}/'

# Get page content
page = requests.get(url)
year_soup = BeautifulSoup(page.content, 'html.parser')

In [92]:
reload(projections)
data = projections.GetData(year_soup=year_soup)

In [89]:
# data.team_standings(season=1999)
# data.team_links()

,id,division,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,sos_total,srs_total,srs_offense,srs_defense
Indianapolis Colts,clt,AFC East,13,3,.813,423,333,90,5.6,0.5,6.1,6.6,-0.5
Buffalo Bills,buf,AFC East,11,5,.688,320,229,91,5.7,1.5,7.1,-0.2,7.4
Miami Dolphins,mia,AFC East,9,7,.563,326,336,-10,-0.6,2.5,1.9,1.2,0.7
New York Jets,nyj,AFC East,8,8,.500,308,309,-1,-0.1,3.1,3.1,0.2,2.9
New England Patriots,nwe,AFC East,8,8,.500,299,284,15,0.9,1.3,2.2,-1.0,3.2
Jacksonville Jaguars,jax,AFC Central,14,2,.875,396,217,179,11.2,-4.8,6.4,1.3,5.1
Tennessee Titans,oti,AFC Central,13,3,.813,392,324,68,4.3,-3.2,1.0,2.0,-1.0
Baltimore Ravens,rav,AFC Central,8,8,.500,324,277,47,2.9,-2.0,0.9,-1.6,2.5
Pittsburgh Steelers,pit,AFC Central,6,10,.375,317,320,-3,-0.2,-2.2,-2.4,-2.2,-0.2
Cincinnati Bengals,cin,AFC Central,4,12,.250,283,460,-177,-11.1,-0.5,-11.5,-3.3,-8.2
